In [1]:
# importiere Library und build Instanz

from camera_roboter import CameraRoboter
from math import pi
import rospy

CameraRoboterI = CameraRoboter(load_camera=True)
step_length = 0.016
num_step = 2

[ INFO] [1687360186.323803123]: Loading robot model 'panda'...
[ INFO] [1687360187.386985701]: Ready to take commands for planning group panda_arm.
Initialize workspace...
============ Planning frame: panda_link0
============ End effector link: panda_link8
============ Available Planning Groups: ['panda_arm']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  name: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  position: [-2.8086262807718545e-05, -0.786022968248237, 0.00022283485729714992, -2.354747486435507, -0.00046250441208123494, 1.571913338859876, -0.7847755370636781]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_obj

In [2]:
# prüfe scharfes bild und Position
CameraRoboterI.Camera.set_camera_exposure(exposure_ms=15)

try:
    CameraRoboterI.Camera.init_camera()
    CameraRoboterI.move_ready_pose()
    CameraRoboterI.get_video_stream()
finally:
    CameraRoboterI.Camera.close()

# labeln jeder Stelle der Schweißnah

try:
    CameraRoboterI.Camera.init_camera()
    CameraRoboterI.calibration(step_length, num_step)
finally:
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()

initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
StopLiveVideo returns 0
ExitCamera returns 0
initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
StopLiveVideo returns 0
ExitCamera returns 0


In [21]:
# eingebe Info der Probe

dict_info = dict()

dict_info['id'] = "KV1.0mm(V008)"
dict_info['speed'] = "1500"
dict_info['rotate_speed'] = "2900"
dict_info['force'] = "9500"
dict_info['date'] = "01.12.21"

CameraRoboterI.DataManager.set_basic_info(dict_info)
dict_info

{'id': 'KV1.0mm(V008)',
 'speed': '1500',
 'rotate_speed': '2900',
 'force': '9500',
 'date': '01.12.21'}

In [22]:
# Hauptprogramm

try:
    # initialize Kamera
    CameraRoboterI.Camera.init_camera()

    # setzt Heilligkeitstufe
    brightness_class = [30, 50, 70, 100] 

    # jeder Heilligkeitstufe kommplette Prozess laufen
    for i in range(len(brightness_class)):

        # ---manuell Heilligkeit einstellen---
        _ = input("Bitte stellt Heilligkeit manuell auf {} ein!".format(brightness_class[i]))

        # ---linear Bewegung---
        CameraRoboterI.DataManager.set_brightness(brightness_class[i])
        CameraRoboterI.move_ready_pose(save_image=True)
        CameraRoboterI.move_linear_trajectory(step_length, num_step, save_image=True)

        # ---drehe um z-Achse---
        ## kleine Winkel von pi/100
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(pi/100)
        [j0, j1, j2, j3, j4, j5, j6] = [-0.0009979219377353702,
                                        -0.0671110219108431,
                                        -0.0006788550091035156,
                                        -2.3749262930217543,
                                        -0.0005716727840199846,
                                        2.321507068845961,
                                        -0.7557173301876342]
        CameraRoboterI.move_ready_pose()
        CameraRoboterI.move_joint(j0, j1, j2, j3, j4, j5, j6)
        CameraRoboterI.get_image(take_image=True)
        CameraRoboterI.move_linear_trajectory(step_length, num_step, 0.00005, save_image=True)

        ## etwa größe Winkel von pi/50
        [j0, j1, j2, j3, j4, j5, j6] = [-0.002271913000445309,
                                        -0.06699430704339969,
                                        -0.0014601017210513546,
                                        -2.374805681216187,
                                        -0.0005901301697207107,
                                        2.321103171878391,
                                        -0.7259153423806032]
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(pi/50)
        CameraRoboterI.move_ready_pose()
        CameraRoboterI.move_joint(j0, j1, j2, j3, j4, j5, j6)
        CameraRoboterI.get_image(take_image=True)
        CameraRoboterI.move_linear_trajectory(step_length, num_step, 0.00005,save_image=True)

        # setzt Winkel wieder zurück auf Null
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(0)

        # ---drehe um x-Achse--- um pi/25
        [j0, j1, j2, j3, j4, j5, j6] = [0.011400288734650401,
                                        -0.052373508798448654,
                                        0.09629108052504688,
                                        -2.3537710253397623,
                                        -0.16848893251021702,
                                        2.284133459386633,
                                        -0.5647626810855442]
        CameraRoboterI.move_ready_pose()
        angle = CameraRoboterI.DataManager.set_rotate_angle_x(pi/25)
        CameraRoboterI.move_joint(j0, j1, j2, j3, j4, j5, j6)
        CameraRoboterI.get_image(take_image=True)
        CameraRoboterI.move_linear_trajectory(step_length, num_step, -0.0004, save_image=True)

        # setzt Winkel wieder zuruck auf Null
        _ = CameraRoboterI.DataManager.set_rotate_angle_x(0)

        # ---unscharfe manuell einstellen---
        ## manuell einstellen
        if brightness_class[i] == 50:
            CameraRoboterI.move_ready_pose()
            CameraRoboterI.get_video_stream()

            ## linear Bewegung
            CameraRoboterI.DataManager.set_zoom_blur(1)
            CameraRoboterI.move_ready_pose(save_image=True)
            CameraRoboterI.move_linear_trajectory(step_length, num_step, save_image=True)

            # setzt blur wieder zurück
            CameraRoboterI.move_ready_pose()
            CameraRoboterI.get_video_stream()
            CameraRoboterI.DataManager.set_zoom_blur(0)

finally:
    # schließe Kamera und zurück zur Initialposition
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()


initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:12_0.49_30_0_0_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:13_0.51_30_0_0_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:14_0.52_30_0_0_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:16_0.49_30_0_0.03_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:18_0.51_30_0_0.03_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:19_0.52_30_0_0.03_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:21_0.49_30_0_0.06_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:03:22_0.51_30_0_0.06_0.jpg
image

[ERROR] [1687359916.344543692]: Got a callback on a goalHandle that we're not tracking.                 This is an internal SimpleActionClient/ActionClient bug.                 This could also be a GoalID collision


image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:05:17_0.51_100_0.13_0_0.jpg
image is saved als name: images/KV1.0mm(V008)_1500_2900_9500/21.06.2023_17:05:18_0.52_100_0.13_0_0.jpg
StopLiveVideo returns 0
ExitCamera returns 0
